In [1]:
import pandas as pd
import numpy as np
# import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [2]:
pd.set_option('display.max_columns', None)

train = pd.read_csv('train_final.csv', low_memory=False)
validation = pd.read_csv('val_set_final.csv')

In [3]:
train.head()

,Unnamed: 0,date,customer_code,employee_index,country,female,age,new_cust,seniority_in_months,cust_type,residency_spain,birth_spain,join_channel,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,customer_code_encoded,income_to_age
0,0,2015-07-28,664160,N,1,0,0.632653,0,0.402344,1,1,0,KAR,MADRID,0,1.989686,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.424185,1.749698,1.407278,263662,3.144938
1,1,2016-01-28,1076784,N,1,0,0.214286,0,0.152344,1,1,0,KHE,LERIDA,0,-0.306603,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.886876,1.006139,1.407278,459750,-1.430748
2,2,2015-12-28,672465,N,1,0,0.387755,0,0.417969,1,1,0,KFC,SEVILLA,1,-0.148205,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,1,0,1.559984,1.382030,1.407278,266890,-0.382203
3,3,2015-10-28,774528,N,1,0,0.397959,0,0.343750,1,1,0,KFA,MURCIA,1,-0.228531,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,1.850124,1.075147,1.407278,300528,-0.574243
4,4,2016-05-28,569598,N,1,0,0.459184,0,0.496094,1,1,0,KAT,MADRID,1,0.588748,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.942077,1.749698,1.407278,227731,1.282133


In [4]:
validation.head()

,Unnamed: 0.1,Unnamed: 0,date,customer_code,employee_index,country_spain,female,age,first_contract_date,new_cust,seniority_in_months,primary_cust,last_date_primary,cust_type,residency_spain,birth_spain,join_channel,deceased,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,payroll_acct.1,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,income_to_age
0,0,0,2015-11-28,161428,N,1,1,0.744898,2000-03-16,0,0.738281,1,0,1,1,1,KAS,0,MADRID,0,-0.001810,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.015705,1.749698,1.407278,-0.002430
1,1,1,2015-12-28,367478,N,1,1,0.418367,2002-07-02,0,0.632812,1,0,1,1,0,KAT,0,MADRID,1,1.473140,02 - PARTICULARES,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,0,1,0,1.942077,1.749698,1.407278,3.521079
2,2,2,2015-11-28,643150,N,1,0,0.520408,2006-09-21,0,0.429688,1,0,1,1,0,KFC,0,ALBACETE,0,-0.131613,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.559984,1.103044,1.407278,-0.252899
3,3,3,2016-04-28,1385854,N,1,0,0.367347,2015-03-03,0,0.050781,1,0,1,1,0,KHK,0,SEVILLA,0,-0.056218,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.122151,1.382030,1.407278,-0.153034
4,4,4,2015-08-28,495733,N,1,0,0.346939,2004-10-13,0,0.507812,1,0,1,1,0,KFC,0,MADRID,0,0.289349,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1.559984,1.749698,1.407278,0.833982


In [5]:
train = train.drop(columns=['Unnamed: 0'])
validation = validation.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

# Reading into the data

In [6]:
drop = ['join_channel', 'province_name', 'employee_index', 'segment', 'total_products']
train = train.drop(columns=drop + ['customer_code_encoded'])
validation = validation.drop(columns=drop + ['payroll_acct.1', 'first_contract_date', 'primary_cust', 'last_date_primary', 'deceased'])

In [7]:
train = train.rename(columns={'country': 'country_spain'})

In [8]:
products = ['savings_acct', 'guarantees', 'current_acct', 'derivada_acct', 'payroll_acct', 
            'junior_acct', 'mas_particular_acct', 'particular_acct', 'particular_plus_acct', 
            'short_term_depo', 'medium_term_depo', 'long_term_depo', 'e_acct', 'funds', 
            'mortgage', 'pension', 'loans', 'taxes', 'credit_card', 'securities', 
            'home_acct', 'payroll_acct', 'pensions_2', 'direct_debt']

In [9]:
# Check for missing values and duplicates
# train = train.drop_duplicates(subset=['customer_code_encoded'], keep='last') 702609 or sth

In [10]:
print(train.info())
print(validation.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6579717 entries, 0 to 6579716
Data columns (total 42 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   date                    object 
 1   customer_code           int64  
 2   country_spain           int64  
 3   female                  int64  
 4   age                     float64
 5   new_cust                int64  
 6   seniority_in_months     float64
 7   cust_type               int64  
 8   residency_spain         int64  
 9   birth_spain             int64  
 10  active_cust             int64  
 11  income                  float64
 12  savings_acct            int64  
 13  guarantees              int64  
 14  current_acct            int64  
 15  derivada_acct           int64  
 16  payroll_acct            int64  
 17  junior_acct             int64  
 18  mas_particular_acct     int64  
 19  particular_acct         int64  
 20  particular_plus_acct    int64  
 21  short_term_depo         int64  

# Pre-processing

In [11]:
# Store customer IDs separately
customer_ids = train['customer_code'].values

In [12]:
# Split data into features (X) and labels (y)
X_train = train.drop(['customer_code', 'date'] + products, axis=1)
y_train = train[products]

X_val = validation.drop(['customer_code', 'date'] + products, axis=1)
y_val = validation[products]

In [13]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (6579717, 17)
Shape of y_train: (6579717, 24)
Shape of X_val: (2100202, 17)
Shape of y_val: (2100202, 24)


In [14]:
# Store already active products for each customer
already_active = {}
for row in train[['customer_code'] + products].values:
    row = list(row)
    customer_id = row.pop(0)
    active_products = [product for product, value in zip(products, row) if value > 0]
    already_active[customer_id] = active_products

In [15]:
# Hyperparameters
hyperparameter_variations = [
    {'C': 0.01, 'solver': 'liblinear', 'max_iter': 100},
    {'C': 1, 'solver': 'lbfgs', 'max_iter': 200},
    {'C': 10, 'solver': 'liblinear', 'max_iter': 300},
]

In [16]:
# Dictionary to store results
results = {}
id_preds = defaultdict(list)

# Training

In [18]:
# Train and evaluate each variation
for i, params in enumerate(hyperparameter_variations):
    print(f"\nTraining variation {i + 1} with parameters: {params}")
    
    # Create logistic regression model with current parameters
    clf = LogisticRegression(**params)
    
    # Fit the model to the training data for each product
    for product in products:
        # Ensure target is a 1D array/Series
        target = y_train[product]
        # if target.ndim > 1:
        #     target = target.iloc[:, 0]  # Convert to 1D by selecting the first column
        
        # Train the logistic regression model
        clf.fit(X_train, target)
        
        # Predict probabilities on the validation set
        y_val_pred = clf.predict_proba(X_val)[:, 1]  # Take probabilities for the positive class
        
        # Check shape of y_val[product] and y_val_pred for debugging
        print(f"Shape of y_val[product]: {y_val[product].shape}")
        print(f"Shape of y_val_pred: {y_val_pred.shape}")
        
        # Calculate ROC AUC score
        roc_auc = roc_auc_score(y_val[product], y_val_pred)  # Ensure both are 1D arrays
        
        # Store results for the product
        results.setdefault(f'Variation {i + 1}', {})[product] = roc_auc
        print(f"ROC AUC for product '{product}': {roc_auc}")



Training variation 1 with parameters: {'C': 0.01, 'solver': 'liblinear', 'max_iter': 100}
Shape of y_val[product]: (2100202,)
Shape of y_val_pred: (2100202,)
ROC AUC for product 'savings_acct': 0.8000330611127483
Shape of y_val[product]: (2100202,)
Shape of y_val_pred: (2100202,)
ROC AUC for product 'guarantees': 0.5226201579695183
Shape of y_val[product]: (2100202,)
Shape of y_val_pred: (2100202,)
ROC AUC for product 'current_acct': 0.7465143682243353
Shape of y_val[product]: (2100202,)
Shape of y_val_pred: (2100202,)
ROC AUC for product 'derivada_acct': 0.8599193987964853


ValueError: y should be a 1d array, got an array of shape (6579717, 2) instead.

In [19]:
# Print comparison of results
print("\nComparison of Variations:")
for variation, product_results in results.items():
    for product, roc_auc in product_results.items():
        print(f"{variation} - Product: {product}, ROC AUC: {roc_auc}")


Comparison of Variations:
Variation 1 - Product: savings_acct, ROC AUC: 0.8000330611127483
Variation 1 - Product: guarantees, ROC AUC: 0.5226201579695183
Variation 1 - Product: current_acct, ROC AUC: 0.7465143682243353
Variation 1 - Product: derivada_acct, ROC AUC: 0.8599193987964853


# Generate product recommendations

In [21]:
train_preds = {}
for customer_id in customer_ids:
    preds = []
    for i, params in enumerate(hyperparameter_variations):
        clf = LogisticRegression(**params)
        for product in products:
            clf.fit(X_train, y_train[product])
            p = clf.predict_proba(X_train)[:, 1]  # Get prediction probabilities for the entire dataset
            preds.append((product, p[customer_id]))  

    # Sort the products based on predicted probabilities, excluding already active products
    recommended_products = sorted(preds, key=lambda x: x[1], reverse=True)
    recommended_products = [prod for prod, prob in recommended_products if prod not in already_active[customer_id]]

    # Get top 7 product recommendations
    train_preds[customer_id] = recommended_products[:7]

ValueError: y should be a 1d array, got an array of shape (6579717, 2) instead.